In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

# Remplacez ces valeurs par les informations de votre base de données
db_host = "localhost"
db_name = "olist"
db_user = "postgres"
db_password = "admin"

# Créer une connexion avec la base de données en utilisant sqlalchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")


In [2]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_products_dataset.csv", encoding="utf-8")
data.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [3]:
data = data.rename(columns={"product_category_name": "product_category_name_translation"})

In [7]:
data["product_category_name_translation"].fillna("unknown_category", inplace=True)
unique_categories = set(data["product_category_name_translation"].dropna().unique())
existing_categories = pd.read_sql("SELECT product_category_name FROM olist_productcategorynametranslation", engine)
missing_categories = unique_categories - set(existing_categories["product_category_name"].values)

In [8]:
missing_categories

{'unknown_category'}

In [4]:
# unique_categories = set(data["product_category_name_translation"].dropna().unique())
# existing_categories = pd.read_sql("SELECT product_category_name FROM olist_productcategorynametranslation", engine)
# missing_categories = unique_categories - set(existing_categories["product_category_name"].values)

if missing_categories:
    missing_categories_df = pd.DataFrame({"product_category_name": list(missing_categories),
                                          "product_category_name_english": ["unknown"] * len(missing_categories)})
    missing_categories_df.to_sql('olist_productcategorynametranslation', engine, if_exists='append', index=False)

data["product_category_name_translation"].fillna("unknown_category", inplace=True)


In [6]:
data.head()

,product_id,product_category_name_translation,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [5]:
data.to_sql('olist_product', engine, if_exists='append', index=False)


IntegrityError: (psycopg2.errors.ForeignKeyViolation) ERREUR:  une instruction insert ou update sur la table �� olist_product �� viole la contrainte de cl��
��trang��re �� olist_product_product_category_nam_2e719063_fk_olist_pro ��
DETAIL:  La cl�� (product_category_name_translation)=(unknown_category) n'est pas pr��sente dans la table �� olist_productcategorynametranslation ��.

(Background on this error at: https://sqlalche.me/e/20/gkpj)